In [1]:
import tkinter
from tkinter import *
from keras.models import load_model
import sys
import numpy as np

Using TensorFlow backend.


In [2]:
if sys.platform in ("win32", "darwin"):
    from PIL import ImageGrab
else:
    import pyscreenshot as ImageGrab

In [3]:
model = load_model('mnist.h5')
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img.save("test.png")
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

In [4]:
class MainApp(Tk):
    def __init__(self):
        Tk.__init__(self)
        self.title("MNIST classify GUI")
        self.canvas = Canvas(self, width=400, height=400, bg = "white", cursor="cross")
        self.canvas.bind("<B1-Motion>", self.draw_lines)
        self.label = Label(self, text="Waiting...", font=("Helvetica", 24))
        self.canvas.grid(row=0, column=0, pady=2,columnspan=3)
        self.label.grid(row=1, column=0, pady=2,columnspan=3)
        self.classify_button = Button(self, text='Classify', command=self.classify)
        self.clear_button = Button(self, text='Clear', command=self.clear)
        self.quit_button = Button(self, text='Quit', command=self.quit)
        self.classify_button.grid(row=2, column=0, pady=2)
        self.clear_button.grid(row=2, column=1, pady=2 )
        self.quit_button.grid(row=2, column=2, pady=2 )
    def get_canvas(self):
        x = self.winfo_rootx() + self.canvas.winfo_x()
        y = self.winfo_rooty() + self.canvas.winfo_y()
        xx = x + self.canvas.winfo_width()
        yy = y + self.canvas.winfo_height()
        print(x,xx,y,yy)
        return ImageGrab.grab(bbox=(x, y, xx, yy))
    def quit(self):
        self.destroy()
    def clear(self):
        self.canvas.delete("all")
    def classify(self):
        canvas_img = self.get_canvas()
        cl,acc = predict_digit(canvas_img)
        print(str(cl)+', '+ str(int(acc*100))+'%')
        if acc<0.8:
            self.label.configure(text= "Can't Determine")
        else:
            self.label.configure(text= str(cl)+', '+ str(int(acc*100))+'%')
    def draw_lines(self,event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
        

In [5]:
if __name__ == "__main__":
    app = MainApp()
    app.mainloop()


50 452 184 586
7, 32%
96 498 184 586
6, 35%
